# Where are the women artists here? 🖼️
## In this notebook, I analyze how many paintings from MASP (The São Paulo Museum of Art) collection feature women.

In [1]:
# Import libraries
from PIL import Image
Image.MAX_IMAGE_PIXELS = None  # Allow loading images with large pixels - FIX ERROR!!!
from transformers import AutoProcessor, CLIPModel
import requests
import pandas as pd
import numpy as np
import random
from tqdm import tqdm  # Import tqdm for progress bar display during iteration

# Load "paints_links_full.csv"
df = pd.read_csv("paints_links_full.csv")

# Extract relevant columns from the DataFrame
images = df["images"]
titles = df["titles"]
authors = df["authors"]
urls = df["urls"]

# Load the CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Lists to store the results for each image
results = []

# Iterate through the DataFrame using iterrows()
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing..."):
    image_url = row["images"]  # Extract the image URL from the current row
    title = row["titles"]  # Extract the title from the current row
    author = row["authors"]  # Extract the author from the current row
    url = row["urls"]  # Extract the URL from the current row
    
    try:
        # Request the image data from the provided URL
        response = requests.get(image_url, stream=True)
        response.raise_for_status()  # Check if the response returned a 2xx status code (success)
        image = Image.open(response.raw)  # Open the image using the PIL library
        
        # Prepare inputs for CLIP model, combining text and image data
        inputs = processor(
            text=[
                "In this painting, there is a woman", 
                "In this painting, there is a man",
                "In this painting, there are many people", 
                "In this painting, there are no people"
                 ],
            images=image,
            return_tensors="pt",
            padding=True
        )
            
        outputs = model(**inputs)  # Get the CLIP model outputs for the combined data
        logits_per_image = outputs.logits_per_image  # These are the similarity scores between image and text
        probs = logits_per_image.softmax(dim=1)  # Use softmax to get label probabilities
        
        # Condition to add the results of this image to the 'results' list
        if not pd.isnull(title) and not pd.isnull(author) and not pd.isnull(url):
            results.append({"image": image_url, "title": title, "author": author, "url": url, "probs": probs.tolist()})
        
    except requests.exceptions.RequestException as e:
        print(f"The image URL is {image_url}") # Look errors

Processing...:  39%|██████████▏               | 174/442 [02:16<02:47,  1.60it/s]

The image URL is https://assets.masp.org.br/uploads/collection/cp7gRe5hCdPvAEwDL1V6JlgVpP42T8Ml.jpg


Processing...:  40%|██████████▎               | 176/442 [02:18<03:02,  1.45it/s]

The image URL is https://assets.masp.org.br/uploads/collection/085pHfgpOjazwYJkG3osRS0SHvCMAf5U.jpg


Processing...: 100%|██████████████████████████| 442/442 [05:07<00:00,  1.44it/s]


## Ok, now I need to select 50 random images and analyze them using my eyes (and my brain 🧠) 

In [2]:
# Get the indices of 50 random items from the results
random_selection = random.sample(range(len(results)), 50)

In [3]:
# Create a list to store the randomly selected 50 items 🎲
random_results = []

In [4]:
# Create a list to store the randomly selected 50 items
for idx in random_selection:
    random_results.append(results[idx])

## A little study for the boring part of this review 😴
##### OBS: I asked ChatGPT to fix this in a prettier way

1.
- Image: "https://assets.masp.org.br/uploads/collection/c9GzOvnmPaXXktDDsCtraXvhbppbEgr3.jpg"
- Title: "Através do espelho mágico (Pietro Maria Bardi), 1987"
- Author: "Wesley Duke Lee"
- URL: "https://masp.org.br/acervo/obra/atraves-do-espelho-magico-pietro-maria-bardi"
- Probs: [43.53%, 33.66%, 10.93%, 11.87%]
- My Analysis: man
- Conclusion: ERROR

2.
- Image: "https://assets.masp.org.br/uploads/collection/Iuid9KFRhb98Fd80GpJFgZJGn0o8ZZlR.jpg"
- Title: "Ernst Christian Wilhelm Bruch, Século 19"
- Author: "Desconhecido (Pintor alemão)"
- URL: "https://masp.org.br/en/collections/works/ernst-christian-wilhelm-bruch-1"
- Probs: [17.06%, 68.88%, 5.65%, 8.41%]
- My Analysis: man
- Conclusion: GREAT!!!

3.
- Image: "https://assets.masp.org.br/uploads/collection/U4NYiuYD6AzTWcATB8CSUz7H1MtI2EtD.jpg"
- Title: "Flagelação de um negro, 1948"
- Author: "José Antônio da Silva"
- URL: "https://masp.org.br/acervo/obra/flagelacao-de-um-negro"
- Probs: [5.38%, 34.17%, 25.98%, 34.47%]
- My Analysis: many people
- Conclusion: GREAT!!!

4.
- Image: "https://assets.masp.org.br/uploads/collection/h0wZeZaDL8LG2fZfjldHpHJXiPFbz0NL.jpg"
- Title: "Dona Tereza Cristina, 1864"
- Author: "Victor Meirelles"
- URL: "https://masp.org.br/acervo/obra/dona-tereza-cristina"
- Probs: [76.19%, 10.83%, 7.85%, 5.13%]
- My Analysis: woman
- Conclusion: GREAT!!!

5.
- Image: "https://assets.masp.org.br/uploads/collection/IWevaJPfNI4DpVR4rsaykGSyY0LP3Aaz.jpg"
- Title: "Niterói, 1889"
- Author: "Nicolau Facchinetti"
- URL: "https://masp.org.br/acervo/obra/niteroi"
- Probs: [19.11%, 17.50%, 50.63%, 12.77%]
- My Analysis: man
- Conclusion: ERROR

6.
- Image: "https://assets.masp.org.br/uploads/collection/kzE5x7nav8PPJGGutXxJqD3MC3U4mkQj.jpg"
- Title: "Marinha, 1951"
- Author: "José Pancetti"
- URL: "https://masp.org.br/acervo/obra/marinha"
- Probs: [14.23%, 10.66%, 70.55%, 4.57%]
- My Analysis: no people
- Conclusion: GREAT!!!

7.
- Image: "https://assets.masp.org.br/uploads/collection/epbrL5CbzdHsrOef64S9BsxgZDQpOIOg.jpg"
- Title: "Interior de laboratório, 1903"
- Author: "José Júlio de Souza Pinto"
- URL: "https://masp.org.br/acervo/obra/interior-de-laboratorio"
- Probs: [27.98%, 21.40%, 28.66%, 21.96%]
- My Analysis: no people
- Conclusion: GREAT!!!

8.
- Image: "https://assets.masp.org.br/uploads/collection/NI7yCAgsqWzBrnNuRE5DQL1jFjnagGSR.jpg"
- Title: "A educação faz tudo, 1775-80"
- Author: "Jean-Honoré Fragonard"
- URL: "https://masp.org.br/acervo/obra/a-educacao-faz-tudo"
- Probs: [44.09%, 20.53%, 16.69%, 18.70%]
- My Analysis: many people
- Conclusion: ERROR

9.
- Image: "https://assets.masp.org.br/uploads/collection/PteRTtMfjPVTuJkLylfgwbavrVbryzol.jpg"
- Title: "Paisagem nos arredores do Rio de Janeiro, 1864"
- Author: "Henri Nicolas Vinet"
- URL: "https://masp.org.br/acervo/obra/paisagem-nos-arredores-do-rio-de-janeiro"
- Probs: [12.01%, 19.21%, 48.58%, 20.20%]
- My Analysis: many people
- Conclusion: ERROR

10.
- Image: "https://assets.masp.org.br/uploads/collection/pTrsIjSgLSQeYAwgFrq2D9AfHt8P58X1.jpg"
- Title: "Sacré-Coeur de Montmartre e Château des Brouillards, 1934"
- Author: "Maurice Utrillo"
- URL: "https://masp.org.br/acervo/obra/sacre-coeur-de-montmartre-e-chateau-des-brouillards"
- Probs: [19.59%, 21.64%, 37.84%, 20.92%]
- My Analysis: many people
- Conclusion: ERROR

11.
- Image: "https://assets.masp.org.br/uploads/collection/yTXpcOc1Mk7hrFaV2lI4S3UwwXvYxbXZ.jpg"
- Title: "O ateliê do artista, 1894"
- Author: "José Malhoa"
- URL: "https://masp.org.br/acervo/obra/o-atelie-do-artista-1"
- Probs: [70.14%, 15.92%, 10.14%, 3.80%]
- My Analysis: many people
- Conclusion: ERROR

12.
- Image: "https://assets.masp.org.br/uploads/collection/s2QzGH0rgdlYd4PDUJZbd4dAR6cP5zsJ.jpg"
- Title: "Madame Marie-Adélaïde de France – O ar, 1751"
- Author: "Jean-Marc Nattier"
- URL: "https://masp.org.br/acervo/obra/madame-marie-adelaide-de-france-o-ar"
- Probs: [72.24%, 9.34%, 10.42%, 7.99%]
- My Analysis: woman
- Conclusion: GREAT!!!

13.
- Image: "https://assets.masp.org.br/uploads/collection/ajTF4SbSv0uinF9u3fj3mndSU8TNG67d.jpg"
- Title: "O escolar (O filho do carteiro – Gamin au Képi), 1888"
- Author: "Vincent van Gogh"
- URL: "https://masp.org.br/acervo/obra/o-escolar-o-filho-do-carteiro-gamin-au-kepi"
- Probs: [48.83%, 26.67%, 20.26%, 4.23%]
- My Analysis: man
- Conclusion: ERROR

14.
- Image: "https://assets.masp.org.br/uploads/collection/JCZTh9PUcG5YQz3hbZH6QxPvRMR1QrMv.jpg"
- Title: "Banhistas no Sena – Academia, 1874-76"
- Author: "Édouard Manet"
- URL: "https://masp.org.br/acervo/obra/banhistas-no-sena-academia"
- Probs: [78.26%, 13.15%, 6.28%, 2.32%]
- My Analysis: many people
- Conclusion: ERROR

15.
- Image: "https://assets.masp.org.br/uploads/collection/otJgk8y1K9gMYrpPdLFUo8G2CnmmNLHS.jpg"
- Title: "Retrato de jovem com corrente de ouro (autorretrato com corrente de ouro), circa 1635"
- Author: "Rembrandt e Ateliê"
- URL: "https://masp.org.br/acervo/obra/retrato-de-jovem-com-corrente-de-ouro-autorretrato-com-corrente-de-ouro"
- Probs: [20.15%, 56.19%, 17.56%, 6.10%]
- My Analysis: man
- Conclusion: GREAT!!!

16.
- Image: "https://assets.masp.org.br/uploads/collection/E8l7KeVAzZPDSD3lM4lZxed8vhTjYeN4.jpg"
- Title: "Pós-escrito – A guerra secreta de Josephine Baker, Sem data"
- Author: "R. B. Kitaj"
- URL: "https://masp.org.br/acervo/obra/postscript-the-secret-war-of-josephine-baker"
- Probs: [20.70%, 26.53%, 44.26%, 8.51%]
- My Analysis: no people
- Conclusion: GREAT!!!

17.
- Image: "https://assets.masp.org.br/uploads/collection/yMGcYs7Jzj1uEkAaww79ytLVWIaAlkt8.jpg"
- Title: "Retrato de Gianfranco Contini aos dezesseis anos e uma marinha, 1928"
- Author: "Filippo de Pisis"
- URL: "https://masp.org.br/acervo/obra/retrato-de-gianfranco-contini-aos-dezesseis-anos-e-uma-marinha"
- Probs: [16.72%, 59.45%, 20.30%, 3.53%]
- My Analysis: man
- Conclusion: GREAT!!!

18.
- Image: "https://assets.masp.org.br/uploads/collection/ZZQeTSm777y78brYBsVi3yZ8JzjLFmc3.jpg"
- Title: "Projeto de vitral, 1963"
- Author: "John Piper"
- URL: "https://masp.org.br/acervo/obra/projeto-de-vitral"
- Probs: [13.91%, 12.61%, 40.74%, 32.73%]
- My Analysis: no people
- Conclusion: GREAT!!!

19.
- Image: "https://assets.masp.org.br/uploads/collection/tThV4ZlEds9NU3bYvkHzxaEct3vK6Jqp.jpg"
- Title: "Lindo lindo lindo, 1976"
- Author: "José Antônio da Silva"
- URL: "https://masp.org.br/acervo/obra/lindo-lindo-lindo"
- Probs: [3.52%, 7.60%, 61.76%, 27.11%]
- My Analysis: no people
- Conclusion: GREAT!!!

20.
- Image: "https://assets.masp.org.br/uploads/collection/LXtaU1wG7tvkMZkSd9S9qwlDBoPYAaIH.jpg"
- Title: "Mulher se penteando – Duas mulheres em camisola, 1891"
- Author: "Henri de Toulouse-Lautrec"
- URL: "https://masp.org.br/acervo/obra/mulher-se-penteando-duas-mulheres-em-camisola"
- Probs: [61.19%, 17.25%, 11.38%, 10.19%]
- My Analysis: many people
- Conclusion: ERROR

21.
- Image: "https://assets.masp.org.br/uploads/collection/xE6y7yz5L4FHaACWunFlDAdXSLsyS6D0.jpg"
- Title: "Nu feminino, 1930-33"
- Author: "Pierre Bonnard"
- URL: "https://masp.org.br/acervo/obra/nu-feminino"
- Probs: [83.77%, 8.28%, 4.65%, 3.30%]
- My Analysis:woman
- Conclusion: GREAT!!!

22.
- Image: "https://assets.masp.org.br/uploads/collection/t3EFPNs753QjIJc8Ioij85exl4RUtDJ8.jpg"
- Title: "Angústia (a mãe do artista), 1950"
- Author: "David Alfaro Siqueiros"
- URL: "https://masp.org.br/acervo/obra/angustia-a-mae-do-artista"
- Probs: [63.27%, 16.61%, 12.84%, 7.28%]
- My Analysis: woman
- Conclusion: GREAT!!!

23.
- Image: "https://assets.masp.org.br/uploads/collection/jDDpF4u0NTXxfmmwYgXJss3yqpGyT106.jpg"
- Title: "Lunia Czcchwska, circa 1918"
- Author: "Amedeo Modigliani"
- URL: "https://masp.org.br/acervo/obra/lunia-czcchwska-"
- Probs: [82.54%, 10.49%, 4.09%, 2.89%]
- My Analysis: woman
- Conclusion: GREAT!!!

24.
- Image: "https://assets.masp.org.br/uploads/collection/gB4xveFHkLbxJ0pioGV4iRhJZ8RJrN64.jpg"
- Title: "Panorama da Baía de Guanabara, 1825"
- Author: "Maria Graham"
- URL: "https://masp.org.br/acervo/obra/panorama-da-baia-de-guanabara"
- Probs: [16.21%, 13.12%, 53.80%, 16.86%]
- My Analysis: no people
- Conclusion: GREAT!!!

25.
- Image: "https://assets.masp.org.br/uploads/collection/SomLg3Bz0AHo4nxCb8gLszTW1gYTxeEx.jpg"
- Title: "Johann Christian Bruch, circa 1771"
- Author: "Desconhecido (Pintor alemão)"
- URL: "https://masp.org.br/acervo/obra/johann-christian-bruch"
- Probs: [31.83%, 38.30%, 17.73%, 12.14%]
- My Analysis: man
- Conclusion: GREAT!!!

26.
- Image: "https://assets.masp.org.br/uploads/collection/Ca49jbkfpmxwMsCEHOngQkLzFMCRcOdL.jpg"
- Title: "O jagunço, 1967"
- Author: "Aldemir Martins"
- URL: "https://masp.org.br/acervo/obra/o-jagunco"
- Probs: [28.46%, 42.14%, 20.13%, 9.27%]
- My Analysis: man
- Conclusion: GREAT!!!

27.
- Image: "https://assets.masp.org.br/uploads/collection/uQXinmIClsiKwDyOk8ONvrlBhNErlVdr.jpg"
- Title: "Marte e Vênus com uma roda de cupidos e paisagem, 1605-10"
- Author: "Carlo Saraceni"
- URL: "https://masp.org.br/acervo/obra/marte-e-venus-com-uma-roda-de-cupidos-e-paisagem"
- Probs: [34.84%, 26.95%, 21.54%, 16.68%]
- My Analysis: many people
- Conclusion: ERROR

28.
- Image: "https://assets.masp.org.br/uploads/collection/lQGqxbuIBEsYPKNoNzd7ktspI2rdMsVI.jpg"
- Title: "O menino, 1917"
- Author: "Arthur Timótheo da Costa"
- URL: "https://masp.org.br/acervo/obra/o-menino"
- Probs: [31.30%, 30.54%, 27.52%, 10.63%]
- My Analysis: man
- Conclusion: ERROR

29.
- Image: "https://assets.masp.org.br/uploads/collection/Nqgfy9HFN4YyFr3mnZRRuFdYbbGGDiAS.jpg"
- Title: "Madame Cézanne em vermelho, 1888-90"
- Author: "Paul Cézanne"
- URL: "https://masp.org.br/acervo/obra/madame-cezanne-em-vermelho"
- Probs: [86.32%, 9.79%, 2.65%, 1.24%]
- My Analysis: woman
- Conclusion: GREAT!!!

30.
- Image: "https://assets.masp.org.br/uploads/collection/MvP6uqhSdVga5XqKYuvZCEh0pA5DapXL.jpg"
- Title: "Paisagem urbana, 1929"
- Author: "Rudolf Jacobi"
- URL: "https://masp.org.br/acervo/obra/paisagem-urbana"
- Probs: [14.34%, 13.68%, 28.49%, 43.50%]
- My Analysis: many people
- Conclusion: GREAT!!!

31.
- Image: "https://assets.masp.org.br/uploads/collection/HDZiTBDbqkXTlUb8vVdvfeBuuNy1ljZQ.jpg"
- Title: "Menina com as espigas, 1888"
- Author: "Pierre-Auguste Renoir"
- URL: "https://masp.org.br/acervo/obra/menina-com-as-espigas"
- Probs: [50.45%, 13.09%, 14.14%, 22.31%]
- My Analysis: woman
- Conclusion: GREAT!!!

32.
- Image: "https://assets.masp.org.br/uploads/collection/xoK0v8m5YencZ3da0rjHTjGsKY3Quh68.jpg"
- Title: "Retrato de Elisabeth-Sophie-Constance de Lowendahl, condessa de Turpin de Crissé, 1775-85"
- Author: "Jean-Honoré Fragonard"
- URL: "https://masp.org.br/acervo/obra/retrato-de-elisabeth-sophie-constance-de-lowendahl-condessa-de-turpin-de-crisse"
- Probs: [88.24%, 4.69%, 3.67%, 3.41%]
- My Analysis: woman
- Conclusion: GREAT!!!

33.
- Image: "https://assets.masp.org.br/uploads/collection/ZRG2nxW0MzoFMkp3o73ljOLC5cTZoA7B.jpg"
- Title: "Cena de café (retrato de Benito Mussolini?), Sem data"
- Author: "John Henri Deluc"
- URL: "https://masp.org.br/acervo/obra/cena-de-cafe-retrato-de-benito-mussolini"
- Probs: [8.16%, 21.01%, 32.41%, 38.42%]
- My Analysis: many people
- Conclusion: GREAT!!!

34.
- Image: "https://assets.masp.org.br/uploads/collection/Zm1dJviLS7aPJx0YIirrPVR208i8GFKo.jpg"
- Title: "Anunciação, circa 1600"
- Author: "El Greco"
- URL: "https://masp.org.br/acervo/obra/anunciacao"
- Probs: [63.63%, 15.35%, 8.96%, 12.07%]
- My Analysis: many people
- Conclusion: ERROR

35.
- Image: "https://assets.masp.org.br/uploads/collection/qHXjxYcCPJ4NqcAV2AcrNysyTCc5rnzW.jpg"
- Title: "Índios Xavantes na Missão São Marcos, 1976"
- Author: "Sepp Baendereck"
- URL: "https://masp.org.br/acervo/obra/indios-xavantes-na-missao-sao-marcos"
- Probs: [17.30%, 26.58%, 27.42%, 28.69%]
- My Analysis: many people
- Conclusion: GREAT!!!

36.
- Image: "https://assets.masp.org.br/uploads/collection/2akqU2g8O3T0MRQesvxJDgvrEqS44pTd.jpg"
- Title: "Natureza-morta com morangos, Sem data"
- Author: "Pedro Alexandrino"
- URL: "https://masp.org.br/acervo/obra/natureza-morta-com-morangos"
- Probs: [18.92%, 24.43%, 40.81%, 15.84%]
- My Analysis: no people
- Conclusion: GREAT!!!

37.
- Image: "https://assets.masp.org.br/uploads/collection/gS8TGCNXYuo0Gds49AoOeoDtSutDij5x.jpg"
- Title: "Kongara-Doji (acólito de Fudo Myo-o), Período Edo Inicial"
- Author: "Civilização japonesa"
- URL: "https://masp.org.br/acervo/obra/kongara-doji-acolito-de-fudo-myo-o"
- Probs: [50.27%, 20.48%, 18.42%, 10.83%]
- My Analysis: man
- Conclusion: ERROR

38.
- Image: "https://assets.masp.org.br/uploads/collection/msEArbvLdGE4KwS435vpuxBhidqxxcGt.jpg"
- Title: "Paisagem da Bretanha (Village Breton Belle-Île), 1897-98"
- Author: "Henri Matisse"
- URL: "https://masp.org.br/acervo/obra/paisagem-da-bretanha-village-breton-belle-ile"
- Probs: [14.02%, 23.76%, 43.29%, 18.93%]
- My Analysis: many people
- Conclusion: ERROR

39.
- Image: "https://assets.masp.org.br/uploads/collection/sjNgRyqoUflTaL57TRgdzFbi8nr0dqWz.jpg"
- Title: "Igreja em Terrassa, 1914-19"
- Author: "Joaquín Torres-García"
- URL: "https://masp.org.br/acervo/obra/igreja-em-terrassa"
- Probs: [8.11%, 16.94%, 59.56%, 15.39%]
- My Analysis: no people
- Conclusion: GREAT!!!

40.
- Image: "https://assets.masp.org.br/uploads/collection/JtYHw63mHkC6yf7F8eCmyabyG6FZpXGx.jpg"
- Title: "A senhora Aimée, da série Os Retratos, 1945"
- Author: "Candido Portinari"
- URL: "https://masp.org.br/acervo/obra/a-senhora-aimee-da-serie-os-retratos"
- Probs: [85.56%, 6.36%, 3.63%, 4.45%]
- My Analysis: woman
- Conclusion: GREAT!!!

41.
- Image: "https://assets.masp.org.br/uploads/collection/OB0luosge285trbUedkJjtRy2ainu6Tp.jpg"
- Title: "O outono – Baco e Ariadne, 1856-63"
- Author: "Eugène Delacroix"
- URL: "https://masp.org.br/acervo/obra/o-outono-baco-e-ariadne"
- Probs: [16.98%, 57.84%, 15.68%, 9.50%]
- My Analysis: many people
- Conclusion: ERROR

42.
- Image: "https://assets.masp.org.br/uploads/collection/WQv0bzGFQtEqxvEJENcqkbyo1uF9KRoi.jpg"
- Title: "O grande pinheiro, 1890-96"
- Author: "Paul Cézanne"
- URL: "https://masp.org.br/acervo/obra/o-grande-pinheiro"
- Probs: [12.83%, 26.06%, 43.18%, 17.93%]
- My Analysis: no people
- Conclusion: GREAT!!!

43.
- Image: "https://assets.masp.org.br/uploads/collection/6XXPZZLFLxLVTcizsNDCSTgLpTuJj9zl.jpg"
- Title: "Cipião, 1866-68"
- Author: "Paul Cézanne"
- URL: "https://masp.org.br/acervo/obra/cipiao"
- Probs: [26.74%, 43.40%, 24.78%, 5.08%]
- My Analysis: man
- Conclusion: GREAT!!!

44.
- Image: "https://assets.masp.org.br/uploads/collection/xbXdrS5XT1Hz3dl7n9M1icUA8WRy25CJ.jpg"
- Title: "Pedra Mãe d'Água (paisagem da Tijuca), circa 1850"
- Author: "Félix Émile Taunay"
- URL: "https://masp.org.br/acervo/obra/pedra-mae-dagua-paisagem-da-tijuca"
- Probs: [17.17%, 21.13%, 53.81%, 7.88%]
- My Analysis: no people
- Conclusion: GREAT!!!

45.
- Image: "https://assets.masp.org.br/uploads/collection/gNaYJS2A22A80hCiTnho7UtpnNkhZqCp.jpg"
- Title: "Natureza-morta com vaso e frutas, 1895"
- Author: "Pedro Alexandrino"
- URL: "https://masp.org.br/acervo/obra/natureza-morta-com-vaso-e-frutas"
- Probs: [60.77%, 22.15%, 11.27%, 5.81%]
- My Analysis: no people
- Conclusion: ERROR

46.
- Image: "https://assets.masp.org.br/uploads/collection/PJxrqfknVynppHBEpVKxzHCwqaX9ZRky.jpg"
- Title: "Rosa e lilás, 1890-1900"
- Author: "Dominique Hubert Rozier"
- URL: "https://masp.org.br/acervo/obra/rosa-e-lilas"
- Probs: [38.26%, 23.33%, 12.70%, 25.71%]
- My Analysis: no people
- Conclusion: ERROR

47.
- Image: "https://assets.masp.org.br/uploads/collection/GacBd1jcuNtky7ep6TYFVscnudPonyAm.jpg"
- Title: "Retrato de Octave Raquin, 1901"
- Author: "Henri de Toulouse-Lautrec"
- URL: "https://masp.org.br/acervo/obra/retrato-de-octave-raquin"
- Probs: [19.24%, 67.09%, 10.71%, 2.96%]
- My Analysis: man
- Conclusion: GREAT!!!

48.
- Image: "https://assets.masp.org.br/uploads/collection/l7UJ9OpSTl2lTIycqw0PfqsS1eyt7DPK.jpg"
- Title: "Oficial sentado, 1631"
- Author: "Frans Hals"
- URL: "https://masp.org.br/acervo/obra/oficial-sentado"
- Probs: [24.70%, 52.71%, 18.78%, 3.82%]
- My Analysis: man
- Conclusion: GREAT!!!

49.
- Image: "https://assets.masp.org.br/uploads/collection/9v8ldZU8qImMFKDLt1HzMJ8sZ1tBoc1e.jpg"
- Title: "Paisagem de Ouro Preto, 1876-86"
- Author: "Henrique Bernardelli"
- URL: "https://masp.org.br/acervo/obra/paisagem-de-ouro-preto"
- Probs: [23.42%, 13.01%, 29.74%, 33.83%]
- My Analysis: no people
- Conclusion: ERROR

50.
- Image: "https://assets.masp.org.br/uploads/collection/qXPWeGoLHxGNG5MtVX8UJl5DTCRnonTO.jpg"
- Title: "Banco de pedra no asilo de Saint-Remy, 1889"
- Author: "Vincent van Gogh"
- URL: "https://masp.org.br/acervo/obra/banco-de-pedra-no-asilo-de-saint-remy"
- Probs: [7.24%, 12.63%, 70.04%, 10.09%]
- My Analysis: no people
- Conclusion: GREAT!!!

## 18 ERROS VS. 32 GREATS!!! 📊

Got 64% correct answers.  
In 440 items, 64% represents ~282 correct answers.

## Saving the results

In [5]:
# ...

# Function to determine the result based on probabilities and return the relative probability
def get_result(probs):
    # Compare probabilities to classify the result
    if probs[0] > probs[1] and probs[0] > probs[2] and probs[0] > probs[3]:
        return "woman", probs[0]  # Return the result and the probability of "woman"
    elif probs[1] > probs[0] and probs[1] > probs[2] and probs[1] > probs[3]:
        return "man", probs[1]  # Return the result and the probability of "man"
    elif probs[2] > probs[0] and probs[2] > probs[1] and probs[2] > probs[3]:
        return "many people", probs[2]  # Return the result and the probability of "many people"
    else:
        return "no_people", probs[3]  # Return the result and the probability of "no_people"

# ...

# Lists to store the information
titles_list = []
authors_list = []
results_list = []
result_probs_list = []  # To store the relative probability
images_list = []

# Process the results
for result in results:
    title = result["title"]
    author = result["author"]
    image_url = result["image"]
    probs = result["probs"][0]
    result_type, result_prob = get_result(probs)  # Separate the result and relative probability
    
    titles_list.append(title)
    authors_list.append(author)
    images_list.append(image_url)  # Append the image URL to the list
    results_list.append(result_type)
    result_probs_list.append(result_prob)  # Append the relative probability to the list

In [6]:
# Construct the df
df_classificated = pd.DataFrame({
    "titles": titles_list,
    "authors": authors_list,
    "result": results_list,
    "percent": result_probs_list,  # Rename the column to store the relative probability
    "images": images_list
})

In [7]:
# Save the new df
df_classificated.to_csv("masp_classificated.csv", index=False)

In [8]:
df_classificated.head()

,titles,authors,result,percent,images
0,"Sra. Franck Rolleston, 1785-92",Gilbert Stuart,woman,0.862348,https://assets.masp.org.br/uploads/collection/...
1,"Vista de Salvador, 1951",Rafael Borjes de Oliveira,many people,0.470691,https://assets.masp.org.br/uploads/collection/...
2,"Cabeça de mulher, 1973",Emiliano Di Cavalcanti,woman,0.783541,https://assets.masp.org.br/uploads/collection/...
3,"Paisagem com um casal de camponeses, Sem data",Eugène Laermans,many people,0.308662,https://assets.masp.org.br/uploads/collection/...
4,"Vaca, Sem data",Émile Claus,no_people,0.385397,https://assets.masp.org.br/uploads/collection/...


In [9]:
# Save all image URLs to a text file
txt_images = df_classificated["images"]

In [11]:
# Export a text file
txt_images.to_csv(r'images.txt', header=None, index=None, sep='\t', mode='a')